In [ ]:
def weather(lon, lat):
    
    # С ПРОГНОЗОМ
    import requests 
    import pandas as pd 
    import json
    from datetime import datetime
    from internal_data import weather_api
    import seaborn as sns
    import matplotlib.pyplot as plt


    # Tomsk city center:
    lat = '56.48'
    lon = '84.98'

    # Sabetta plant area
    # lat = '71.26'
    # lon = '72.05'

    url = f'https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&units={'metric'}&appid={weather_api}'

    response = requests.get(url)

    data = response.json()



    forecast_df = pd.DataFrame(data['hourly'])

    feats_to_convert = ['dt']


    def ts_convert(ts):
        return datetime.fromtimestamp(ts).strftime('%d-%m-%Y %H:%M:%S')

    for feat in feats_to_convert:
        forecast_df[feat] = forecast_df[feat].apply(ts_convert)
        

    def description_extract(data):

        result = f'{data[0]['main']} : {data[0]['description']}'

        return result
        
    forecast_df['info'] = forecast_df['weather'].apply(description_extract)

    forecast_df = forecast_df.drop('weather', axis=1)

    # Преобразование признаков осадков (изначально словарь либо None)
    forecast_df['rain'] = forecast_df['rain'].apply(lambda x: 
        list(x.values())[0] if type(x) is dict else 0
        )

    forecast_df['snow'] = forecast_df['snow'].apply(lambda x: 
        list(x.values())[0] if type(x) is dict else 0
        )



    fig = plt.figure(figsize=(14,6))

    line_graph = sns.lineplot(
        data = forecast_df, 
        x = 'dt',
        y = 'temp'
        )

    line_graph.grid()

    line_graph.set_ylabel('Температура воздуха', fontsize = 16)
    line_graph.set_xlabel('')
    plt.tick_params(axis='x', labelrotation = 90)

    line_title_name = f'Прогноз температуры за период \
        {forecast_df.iloc[0]['dt']} - {forecast_df.iloc[47]['dt']}'

    line_graph.set_title(line_title_name, fontsize = 16) 

In [ ]:
import os

current_folder = os.getcwd()

os_name = os.name

if os_name == 'nt':
    sepr = '\\'
elif os_name == 'posix':
    sepr = '/'
else:
    raise NameError('Different type of OS, unavailble to choose separator')

graph_folder = current_folder + sepr + 'graphs'

In [ ]:
# Проверка на наличие папки под графики
if 'graphs' not in list(os.listdir()):
    
    current_folder = os.getcwd()

    os_name = os.name

    if os_name == 'nt':
        sepr = '\\'
    elif os_name == 'posix':
        sepr = '/'
    else:
        raise NameError('Different type of OS, unavailble to make separator')

    graph_folder = current_folder + sepr + 'graphs'

['.gitignore',
 '01-2025',
 '012025',
 '02-2025',
 '11-2024',
 '12-2024',
 'forecast.ipynb',
 'for_bot.ipynb',
 'history_data.ipynb',
 'internal_data.py',
 'jan_df_csv.csv',
 'jan_df_gooddata.csv',
 'month_extraction.ipynb',
 'snow_forecast.jpg',
 'try_to_read_csv.ipynb',
 '__pycache__']

In [ ]:
#### Блок создания папки под хранения пользовательских графиков

# Путь нахождения программы
path = os.getcwd()

# Название папки под графики, которые будут отправляться пользователю
graph_folder_name = 'graphs'

# Проверка наличие папки под графики
if not os.path.exists(graph_folder_name):
    os.makedirs(graph_folder_name)
else:
    pass

# Проверка имени ОС для выбора нужного разделителя
if os_name == 'nt':
    sepr = '\\'
elif os_name == 'posix':
    sepr = '/'
else:
    raise NameError('Different type of OS, unavailble to make separator')

# Имя папки под графики
graph_folder_path = path + sepr + 'graphs'



In [ ]:
# Блок запроса данных по локации прогноза погоды

import pickle

# Импортируем информацию из файла-хранилища локаций.
# Представляет собой словарь, где ключи - названия, значения - кортежи из двух данных - широта и долгота
with open('locations.pkl', 'rb') as file:
    data = pickle.load(file)

# Запрос на название локации
area_choice = input('Введите, пожаулуйста, название местности')


def new_area(area_choice):
    """Функция создания новой пары ключ-значение в импортируемый словарь data
    с записью полученных значений и сохранению данных

    Args:
        area_choice (str): название локации
    """
    
    area_choice = area_choice
    
    # Информирование об отсутствии локации в данных
    print('Извините, у меня нет гео-данных по этому месту')
    
    # Запрос у пользователя координат нужной локации
    new_coordinates = input('Введите, пожалуйста, широту и долготу этой локации через пробел, например, 56,12 83,10. Не волнуйтесь, каждый раз это делать не придется, я все запомню')
    
    # Создание переменных широты lat и долготы lon
    lat = new_coordinates.split(' ')[0]
    lat = lat.replace(',', '.').strip()
    
    lon = new_coordinates.split(' ')[1]
    lon = lon.replace(',', '.').strip()
    
    # Внесение в словарь новых значений 
    data[area_choice] = (lon, lat)

if area_choice.lower() not in list(data.keys()):
    
    new_area(area_choice)

# Получение данных широты и долготы для нужной локации из словаря 
lon = data[area_choice.lower()][0]
lat = data[area_choice.lower()][1]
    
# Вызов функции прогноза погоды с передачей аргументов широты и долготы в нее
weather(lon, lat)
    
    

Извините, у меня нет гео-данных по этому месту
56.17 45.12


In [9]:
data = {'томск': ('56.13', '58.18')}

with open('locations.pkl', 'wb') as file:
    pickle.dump(data, file)

In [ ]:
import telegram
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
import pickle

# Импорт информации из файла-хранилища локаций
with open('locations.pkl', 'rb') as file:
    data = pickle.load(file)

# Функция обработки команды /start
def start(update, context):
    update.message.reply_text('Привет! Введите название местности для получения прогноза погоды.')

# Функция обработки сообщений от пользователя
def handle_message(update, context):
    area_choice = update.message.text
    
    if area_choice.lower() not in data:
        new_area(area_choice)
    else:
        lon, lat = data[area_choice.lower()]
        weather(lon, lat)

def new_area(area_choice):
    context.bot.send_message(chat_id=update.effective_chat.id, text='Извините, у меня нет гео-данных по этому месту')
    new_coordinates = update.message.text
    
    lat = new_coordinates.split(' ')[0].replace(',', '.').strip()
    lon = new_coordinates.split(' ')[1].replace(',', '.').strip()
    
    data[area_choice.lower()] = (lon, lat)

    with open('locations.pkl', 'wb') as file:
        pickle.dump(data, file)

def weather(lon, lat):
    # Здесь должна быть ваша функция для получения прогноза погоды по широте и долготе

# Инициализация бота и добавление обработчиков команд
updater = Updater("YOUR_TELEGRAM_BOT_TOKEN", use_context=True)
dispatcher = updater.dispatcher

dispatcher.add_handler(CommandHandler('start', start))
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, handle_message))

# Запуск бота
updater.start_polling()
updater.idle()